<a href="https://colab.research.google.com/github/justinacgray/donationProj/blob/main/JGray_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Objective
Predict sales for food items sold at various stores. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in increasing sales.

## Loading Data

In [3]:
filename = "/content/drive/MyDrive/Colab_Notebooks/datasets/sales_predictions.csv"
sales_df = pd.read_csv(filename)
sales_df.info()
sales_df.head()
sales_df_copy= sales_df.copy() # making a copy to refer back to original dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB




1.   There are 12 columns and 8523 rows. 
2.   Columns naming conventions are consistent and Python Syntax. 
3.   There are 4 columns that are floats, 1 column that int, and 7 columns that are objects






In [4]:
sales_df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [5]:
sales_df.shape

(8523, 12)

# Data Cleaning


*   We can use df.duplicated() to show whether rows are duplicates, which will yield True/ False for each row which we don't want
*   We will also use .sum() to add up all of the True values, and True value will evaluate to a 1.




In [6]:
sales_df.duplicated().sum() #there are no reported duplicates! Noice!


0

IF there were any duplicates we use this method to drop them and check again the sum

In [ ]:
# Remove duplicates
# sales_df = sales_df.drop_duplicates()
# sales_df.duplicated().sum()

Now, let's inspect for inconsistent categories of data within our columns with .value_counts() and nunique()

In [7]:
sales_df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

Makes sense for Item_Identifer, Item_Weight, Item_Visibility, Item_Type, Item_MRP, Outlet_Identifier etc to have different unique values. However, the only column that was inconsistent was "Item_Fat_Content. 


In [8]:
# sales_df['Item_Fat_Content'].value_counts() # "Low Fat, Regular, LF, reg, low fat"
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('LF', 'Low Fat')
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('low fat', 'Low Fat')
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('reg', 'Regular')
sales_df['Item_Fat_Content'].value_counts()

# Cleaned!

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

# Identify Missing Values
* WE can use is .isna() to check for missing values but it only tells us for each column True/False
* Therefore, we are going to append .sum() to tally all those True(s)


In [9]:
sales_df.isna().sum() # Item_Wight and Outlet_Size are the only columns where we have nans 

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

How to drop missing values
1. Drop Rows - in this instance too many rows would be dropped and not a good idea. That would be too much data lost
2. Drop Columns - this could be an option alternative to dropping rows for the Outlet_Size column
3. Add New Category - not advised because weight is dependent on the item and outlet size is categorical, and no apparent pattern on how this data was assigned to eacg item
4. Fill in missing data - likely alternative and could fill in the weight with mean and the outlet_size with mode. ~Con~ introducing errors into errors 



In [10]:
median_item_weight = sales_df['Item_Weight'].median()
sales_df['Item_Weight'].fillna(median_item_weight, inplace=True)
sales_df.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [14]:
# mode_outlet_size = sales_df['Outlet_Size'].mode()
# sales_df['Outlet_Size'].fillna(mode_outlet_size, inplace=True)

sales_df.drop(columns='Outlet_Size', inplace=True)
sales_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052


# Summary statistics of each (min, max, mean)
use .describe()

In [23]:
stats = sales_df.describe()
stats.loc[['mean','min','max']]

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
mean,12.81342,0.066132,140.992782,1997.831867,2181.288914
min,4.55500,0.000000,31.290000,1985.000000,33.290000
max,21.35000,0.328391,266.888400,2009.000000,13086.964800
